In [1]:
from ipywidgets import Layout, Button, Box, Checkbox, FloatText, Textarea, Output, Dropdown, Label, IntSlider, GridspecLayout, jslink

In [2]:
import pymmcore
import os.path
import sys
import time
import threading

In [3]:
mm_dir = r"C:\Program Files\Micro-Manager-2.0.1"

mmc = pymmcore.CMMCore()

In [4]:
pymmcore.CMMCore().getAPIVersionInfo()

'Device API version 70, Module API version 10'

In [5]:
mmc.setDeviceAdapterSearchPaths([mm_dir])
mmc.loadSystemConfiguration(os.path.join(mm_dir, "MMConfig_xyz_shutter_filter2.cfg"))

In [6]:
mmc.getLoadedDevices()

('COM1',
 'COM2',
 'COM8',
 'PIZStage',
 'XYStage',
 'SutterHub',
 'Wheel-A',
 'Shutter-A',
 'Core')

# initialize HWP and SYNC

In [7]:
from pylablib.devices import Thorlabs

In [8]:
kinesis_devices = Thorlabs.list_kinesis_devices()
sync_stage, hwp_stage = kinesis_devices[0], kinesis_devices[1]
hwp_ID, sync_ID = kinesis_devices[1][0], kinesis_devices[0][0]
hwp_scale = "stage"
sync_scale = "stage"
print(kinesis_devices)

[('27253344', 'Brushed Motor Controller'), ('27004338', 'Brushed Motor Controller')]


In [9]:
hwp_mutex = threading.Lock()
sync_mutex = threading.Lock()
shutter_mutex = threading.Lock()
filtwhl_mutex = threading.Lock()
xystage_mutex = threading.Lock()
zpiezo_mutex = threading.Lock()
laser_mutex = threading.Lock()

In [10]:
def hwp_home():
    global hwp_mutex

    hwp_mutex.acquire()
    try:
        with Thorlabs.KinesisMotor(hwp_ID, scale="stage") as hwp_stage:
            hwp_stage.home()
            hwp_stage.wait_for_home()
            hwp_stage.move_to(0)
    finally:
        hwp_mutex.release()
        
def sync_home():
    global sync_mutex

    sync_mutex.acquire()
    try:
        with Thorlabs.KinesisMotor(sync_ID, scale="stage") as sync_stage:
            sync_stage.home()
            sync_stage.wait_for_home()
            sync_stage.move_to(0)
    finally:
        sync_mutex.release()

In [54]:
hwp_home()

In [55]:
sync_home()

In [56]:
filter_list = ['Empty', '460/50 NADH', '647/26']

In [74]:
mmc.setShutterOpen(True)
mmc.getShutterOpen()

True

In [78]:
mmc.setShutterOpen(False)
mmc.getShutterOpen()

False

In [81]:
# with Thorlabs.KinesisMotor("27004338", scale="stage") as hwp_stage, Thorlabs.KinesisMotor("27253344", scale="stage") as sync_stage:

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = [
    Box([Label(value="HWP"), FloatText(), FloatText()], layout=form_item_layout),
    Box([Label(value="SYNC"), FloatText(), FloatText()], layout=form_item_layout),
    Box([Label(value="Shutter"), Checkbox(), Checkbox()], layout=form_item_layout),
    Box([Label(value='Emission filter'), Dropdown(options=filter_list), Dropdown(options=filter_list)], layout=form_item_layout),
    Box([Label(value="X"), FloatText(), FloatText()], layout=form_item_layout),
    Box([Label(value="Y"), FloatText(), FloatText()], layout=form_item_layout),
    Box([Label(value="Z"), FloatText(), FloatText()], layout=form_item_layout),
    Box([Label(value="Laser WL TARGET"), FloatText(), FloatText()], layout=form_item_layout),
    Box([Label(value="Laser MTRPOS TARGET"), FloatText(), FloatText()], layout=form_item_layout),
]

def hwp_set_pos(target):
    global hwp_mutex

    hwp_mutex.acquire()
    try:
        with Thorlabs.KinesisMotor(hwp_ID, scale=hwp_scale) as hwp_stage:
            hwp_stage.move_to(target)
            hwp_stage.wait_for_stop()
    finally:
        hwp_mutex.release()
        
def hwp_get_pos():
    global hwp_mutex

    hwp_mutex.acquire()
    try:
        with Thorlabs.KinesisMotor(hwp_ID, scale=hwp_scale) as hwp_stage:
            pos = hwp_stage.get_position()
    finally:
        hwp_mutex.release()
    
    return pos

def sync_set_pos(target):
    global hwp_mutex

    sync_mutex.acquire()
    try:
        with Thorlabs.KinesisMotor(sync_ID, scale=sync_scale) as sync_stage:
            sync_stage.move_to(target)
            sync_stage.wait_for_stop()
    finally:
        sync_mutex.release()
        
def sync_get_pos():
    global sync_mutex

    sync_mutex.acquire()
    try:
        with Thorlabs.KinesisMotor(sync_ID, scale=sync_scale) as sync_stage:
            pos = sync_stage.get_position()
    finally:
        sync_mutex.release()
    
    return pos

def HWP_value_change(change):
    if (change["new"] >= 0) and (change["new"] <= 359.99):        
        HWP_set_pos(change["new"])
        
def SYNC_value_change(change):
    if (change["new"] >= 0) and (change["new"] <= 359.99):        
        SYNC_set_pos(change["new"])
        
def shutter_value_change(change):
    if change["new"]:
        mmc.setShutterOpen(True)
    else:
        mmc.setShutterOpen(False)
        
def filtwhl_value_change(change):
    if (change["new"] >= 0) and (change["new"] <= 359.99):        
        SYNC_set_pos(change["new"])

form_items[0].children[1].observe(HWP_value_change, names="value")
form_items[1].children[1].observe(SYNC_value_change, names="value")
form_items[2].children[1].observe(shutter_value_change, names="value")
form_items[3].children[1].observe(filtwhl_value_change, names="value")


def update_status_bg(form_items):
    """
    Update system status in the background
    """

    total = 100
    for i in range(total):
        time.sleep(0.1)
        form_items[0].children[-1].value = hwp_get_pos()
        form_items[1].children[-1].value = sync_get_pos()
        form_items[2].children[-1].value = mmc.getShutterOpen()

thread = threading.Thread(target=update_status_bg, args=(form_items,))


form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='100%'
))

display(form)
thread.start()

Box(children=(Box(children=(Label(value='HWP'), FloatText(value=0.0), FloatText(value=0.0)), layout=Layout(dis…

# ref

In [ ]:
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = [
    Box([Label(value='HWP'), IntSlider(min=0, max=60)], layout=form_item_layout),
    Box([Label(value='Egg style'),
         Dropdown(options=['Scrambled', 'Sunny side up', 'Over easy'])], layout=form_item_layout),
    Box([Label(value='HWP'), FloatText()], layout=form_item_layout),
    Box([Label(value='Information'),
         Textarea()], layout=form_item_layout)
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))
form